<a href="https://colab.research.google.com/github/yasirabd/solver-society-job-data/blob/main/3_1_Data_cleaning_jobstreet_tahap_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objectives
Data cleaning tahap akhir pada jobstreet tahap akhir. Data output dipersiapkan agar dapat ditampilkan pada Carto.

Data input yang dibutuhkan:
1. jobstreet_clean_tahap3.csv
2. jobstreet_clean_tahap4.csv (Karena proses ekstrak skill lama, dapat gunakan data pertama)
3. company_industry_mapper.csv

Data output yang dihasilkan:
1. data_loker
2. data_pendidikan
3. data_loker_prov
4. data_skill

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [2]:
df = pd.read_csv('/content/drive/My Drive/Data Loker/jobstreet_clean_tahap3.csv')
df.head()

,id_loker,pulau,provinsi,job_position,company_name,years_of_experience,years_of_experience_cat,job_description,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_gaya_berpakaian_contoh,work_environment_tunjangan,work_environment_bahasa,platform,pendidikan
0,1,jawa,dki jakarta,spv finance,PT Axindo Infotama,3,0 - 4 tahun,[SEP][SEP]Usia Max. 35 tahun[SEP]Pendidikan Mi...,6,Retail/Merchandise,51 - 200 pekerja,sabtu,formal,kemeja polos & dasi,"kesehatan, umum",indonesia,jobstreet,d3
1,2,jawa,dki jakarta,qualitative research executive,Perusahaan Dirahasiakan,3,0 - 4 tahun,RESPONSIBILITIES:[SEP]Brief Description of Dut...,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak spesifik,tidak tersedia,tidak dicantumkan,jobstreet,s1
2,3,jawa,jawa tengah,field collection wonogiri,OTO Group,0,0 - 4 tahun,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, bonus",indonesia,jobstreet,"d3, s1, sma"
3,4,jawa,dki jakarta,perawat di negara jerman,PT Jerman Manajemen,1,0 - 4 tahun,[SEP]Candidate must possess at least Diploma i...,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,formal,kemeja polos & dasi,kesehatan,indonesia,jobstreet,d3
4,5,jawa,jawa tengah,field collection pati,OTO Group,0,0 - 4 tahun,[SEP]Lingkup pekerjaan: [SEP]Menjaga account d...,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, bonus",indonesia,jobstreet,"d3, s1, sma"


In [3]:
# drop job_description
df = df.drop(['job_description'], axis=1)

In [4]:
# check null values
df.isnull().sum()

id_loker                                   0
pulau                                      0
provinsi                                   0
job_position                               0
company_name                               0
years_of_experience                        0
years_of_experience_cat                    0
average_processing_time                    0
company_industry                           0
company_size                               0
work_environment_waktu_bekerja             0
work_environment_gaya_berpakaian           0
work_environment_gaya_berpakaian_contoh    0
work_environment_tunjangan                 0
work_environment_bahasa                    0
platform                                   0
pendidikan                                 0
dtype: int64

In [5]:
df.shape

(14036, 17)

In [6]:
data = df.copy()

## Drop row data provinsi = 'tidak diketahui'

In [7]:
# drop row provinsi tidak diketahui
data = data[~data.provinsi.isin(['tidak diketahui'])]
data.shape

(12625, 17)

## Rename nama provinsi

In [8]:
# Ubah nama provinsi supaya sama dengan data Geojson

prov = data['provinsi']
# capitalize first letter
prov = prov.str.title()

# replace dki jakarta
prov = prov.str.replace('Dki Jakarta', 'Daerah Khusus Ibukota Jakarta', regex=False)

# replace Daerah Istimewa Yogyakarta
prov = prov.str.replace('Di Yogyakarta', 'Daerah Istimewa Yogyakarta', regex=False)

prov.value_counts()

Daerah Khusus Ibukota Jakarta    5985
Jawa Barat                       1771
Jawa Timur                       1296
Banten                           1219
Jawa Tengah                       548
Sumatera Utara                    225
Daerah Istimewa Yogyakarta        216
Bali                              214
Kalimantan Timur                  159
Kepulauan Riau                    126
Sulawesi Selatan                  121
Sumatera Selatan                  120
Riau                               86
Kalimantan Selatan                 72
Kalimantan Barat                   54
Kalimantan Tengah                  45
Lampung                            43
Sulawesi Utara                     42
Sulawesi Tengah                    40
Jambi                              37
Nusa Tenggara Barat                37
Kalimantan Utara                   22
Aceh                               21
Sulawesi Tenggara                  20
Nusa Tenggara Timur                18
Sumatera Barat                     17
Papua       

In [9]:
data['provinsi'] = prov

## Capitalize column

In [10]:
# capitalize beberapa column
data['pulau'] = data.pulau.str.title()
data['job_position'] = data.job_position.str.title()
data['work_environment_bahasa'] = data.work_environment_bahasa.str.title()
data['pendidikan'] = data.pendidikan.str.upper()

data.tail()

,id_loker,pulau,provinsi,job_position,company_name,years_of_experience,years_of_experience_cat,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_gaya_berpakaian_contoh,work_environment_tunjangan,work_environment_bahasa,platform,pendidikan
14031,14032,Jawa,Banten,Asisten Notaris,"Kantor Notaris & PPAT Fanny Felicia Chandra, S...",1,0 - 4 tahun,14,Hukum/Legal,1- 50 pekerja,senin - jumat,formal,kemeja polos & dasi,tidak tersedia,Indonesia,jobstreet,TIDAK DICANTUMKAN
14032,14033,Nusa Tenggara,Bali,Client Service And Human Capital Officer,GRATYO® World's Leading Practical Business Coa...,1,0 - 4 tahun,14,Konsultasi (Bisnis & Manajemen),51 - 200 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, umum",Indonesia,jobstreet,"S2, S1"
14033,14034,Jawa,Daerah Khusus Ibukota Jakarta,Demi Chef,Perusahaan Dirahasiakan,4,0 - 4 tahun,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak spesifik,tidak tersedia,Tidak Dicantumkan,jobstreet,D3
14034,14035,Jawa,Daerah Khusus Ibukota Jakarta,Business Development Officer,GRATYO® World's Leading Practical Business Coa...,1,0 - 4 tahun,14,Konsultasi (Bisnis & Manajemen),51 - 200 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, umum",Indonesia,jobstreet,S2
14035,14036,Jawa,Banten,Sales Dan Telemarketing,CV.Orion Metal Indonesia,1,0 - 4 tahun,14,Umum & Grosir,1- 50 pekerja,tidak tersedia,tidak spesifik,tidak spesifik,tidak tersedia,Tidak Dicantumkan,jobstreet,"SMA, D3, S1"


## Reset index dan id_loker

In [11]:
# reset index dan reset id_loker
data = data.reset_index(drop=True)
id_loker = [i+1 for i in range(len(data))]
data['id_loker'] = id_loker

data.tail()

,id_loker,pulau,provinsi,job_position,company_name,years_of_experience,years_of_experience_cat,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_gaya_berpakaian_contoh,work_environment_tunjangan,work_environment_bahasa,platform,pendidikan
12620,12621,Jawa,Banten,Asisten Notaris,"Kantor Notaris & PPAT Fanny Felicia Chandra, S...",1,0 - 4 tahun,14,Hukum/Legal,1- 50 pekerja,senin - jumat,formal,kemeja polos & dasi,tidak tersedia,Indonesia,jobstreet,TIDAK DICANTUMKAN
12621,12622,Nusa Tenggara,Bali,Client Service And Human Capital Officer,GRATYO® World's Leading Practical Business Coa...,1,0 - 4 tahun,14,Konsultasi (Bisnis & Manajemen),51 - 200 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, umum",Indonesia,jobstreet,"S2, S1"
12622,12623,Jawa,Daerah Khusus Ibukota Jakarta,Demi Chef,Perusahaan Dirahasiakan,4,0 - 4 tahun,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak spesifik,tidak tersedia,Tidak Dicantumkan,jobstreet,D3
12623,12624,Jawa,Daerah Khusus Ibukota Jakarta,Business Development Officer,GRATYO® World's Leading Practical Business Coa...,1,0 - 4 tahun,14,Konsultasi (Bisnis & Manajemen),51 - 200 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, umum",Indonesia,jobstreet,S2
12624,12625,Jawa,Banten,Sales Dan Telemarketing,CV.Orion Metal Indonesia,1,0 - 4 tahun,14,Umum & Grosir,1- 50 pekerja,tidak tersedia,tidak spesifik,tidak spesifik,tidak tersedia,Tidak Dicantumkan,jobstreet,"SMA, D3, S1"


## Add jumlah_loker_per_provinsi

In [12]:
# tambah column jumlah_loker_per_provinsi
jumlah_loker_per_provinsi = data.provinsi.value_counts()

# create dictionary
temp = list()
for idx, row in data.iterrows():
    prov = row['provinsi']
    temp.append(jumlah_loker_per_provinsi[prov])

In [13]:
data['jumlah_loker_per_provinsi'] = temp
data.head()

,id_loker,pulau,provinsi,job_position,company_name,years_of_experience,years_of_experience_cat,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_gaya_berpakaian_contoh,work_environment_tunjangan,work_environment_bahasa,platform,pendidikan,jumlah_loker_per_provinsi
0,1,Jawa,Daerah Khusus Ibukota Jakarta,Spv Finance,PT Axindo Infotama,3,0 - 4 tahun,6,Retail/Merchandise,51 - 200 pekerja,sabtu,formal,kemeja polos & dasi,"kesehatan, umum",Indonesia,jobstreet,D3,5985
1,2,Jawa,Daerah Khusus Ibukota Jakarta,Qualitative Research Executive,Perusahaan Dirahasiakan,3,0 - 4 tahun,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak spesifik,tidak tersedia,Tidak Dicantumkan,jobstreet,S1,5985
2,3,Jawa,Jawa Tengah,Field Collection Wonogiri,OTO Group,0,0 - 4 tahun,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, bonus",Indonesia,jobstreet,"D3, S1, SMA",548
3,4,Jawa,Daerah Khusus Ibukota Jakarta,Perawat Di Negara Jerman,PT Jerman Manajemen,1,0 - 4 tahun,11,Pendidikan,51 - 200 pekerja,sabtu - kamis,formal,kemeja polos & dasi,kesehatan,Indonesia,jobstreet,D3,5985
4,5,Jawa,Jawa Tengah,Field Collection Pati,OTO Group,0,0 - 4 tahun,14,Perbankan/Pelayanan Keuangan,lebih dari 5000 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, bonus",Indonesia,jobstreet,"D3, S1, SMA",548


In [14]:
data.tail()

,id_loker,pulau,provinsi,job_position,company_name,years_of_experience,years_of_experience_cat,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_gaya_berpakaian_contoh,work_environment_tunjangan,work_environment_bahasa,platform,pendidikan,jumlah_loker_per_provinsi
12620,12621,Jawa,Banten,Asisten Notaris,"Kantor Notaris & PPAT Fanny Felicia Chandra, S...",1,0 - 4 tahun,14,Hukum/Legal,1- 50 pekerja,senin - jumat,formal,kemeja polos & dasi,tidak tersedia,Indonesia,jobstreet,TIDAK DICANTUMKAN,1219
12621,12622,Nusa Tenggara,Bali,Client Service And Human Capital Officer,GRATYO® World's Leading Practical Business Coa...,1,0 - 4 tahun,14,Konsultasi (Bisnis & Manajemen),51 - 200 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, umum",Indonesia,jobstreet,"S2, S1",214
12622,12623,Jawa,Daerah Khusus Ibukota Jakarta,Demi Chef,Perusahaan Dirahasiakan,4,0 - 4 tahun,tidak tersedia,tidak tersedia,tidak spesifik,tidak tersedia,tidak spesifik,tidak spesifik,tidak tersedia,Tidak Dicantumkan,jobstreet,D3,5985
12623,12624,Jawa,Daerah Khusus Ibukota Jakarta,Business Development Officer,GRATYO® World's Leading Practical Business Coa...,1,0 - 4 tahun,14,Konsultasi (Bisnis & Manajemen),51 - 200 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, umum",Indonesia,jobstreet,S2,5985
12624,12625,Jawa,Banten,Sales Dan Telemarketing,CV.Orion Metal Indonesia,1,0 - 4 tahun,14,Umum & Grosir,1- 50 pekerja,tidak tersedia,tidak spesifik,tidak spesifik,tidak tersedia,Tidak Dicantumkan,jobstreet,"SMA, D3, S1",1219


Note: masih terdapat error perbedaan data setelah export.

In [15]:
data[['id_loker','pendidikan', 'provinsi']].iloc[1312]

id_loker            1313
pendidikan            S1
provinsi      Jawa Timur
Name: 1312, dtype: object

In [16]:
# df_pendidikan[df_pendidikan.id_loker == 1313]

## Clustering industry

In [17]:
# lakukan cluster kelompok berdasarkan Klasifikasi Baku Lapangan Usaha Indonesia
data.company_industry.value_counts()

Manufaktur/Produksi                                     1543
Perbankan/Pelayanan Keuangan                             749
Komputer/Teknik Informatika (Perangkat Lunak)            660
Retail/Merchandise                                       625
Umum & Grosir                                            620
Produk Konsumen/Barang konsumen yang bergerak cepat      549
Manajemen/Konsulting HR                                  527
tidak tersedia                                           486
Properti/Real Estate                                     469
Transportasi/Logistik                                    468
Makanan & Minuman/Katering/Restoran                      454
Konstruksi/Bangunan/Teknik                               442
Kesehatan/Medis                                          400
Agrikultural/Perkebunan/Peternakan Unggas/Perikanan      356
Konsultasi (IT, Ilmu Pengetahuan, Teknis & Teknikal)     347
Asuransi                                                 296
Elektrikal & Elektronik 

Proses dilakukan secara manual pada google sheet.

In [18]:
import re
# export mapping industry
ci_mapper = pd.read_csv("/content/drive/My Drive/Data Loker/company_industry_mapper.csv", header=None)


temp = list()
for idx, row in ci_mapper.iterrows():
    result = ''.join([i for i in row[0] if not i.isdigit()]).strip()
    temp.append(result)

ci_mapper[0] = temp

In [19]:
# create dict
company_industry_mapper = dict()

for idx, row in ci_mapper.iterrows():
    company_industry_mapper[row[0]] = row[1]

In [20]:
data['company_industry'] = data['company_industry'].map(company_industry_mapper).fillna('TIDAK TERSEDIA')
data.company_industry.isnull().sum()

0

In [21]:
data.head()

,id_loker,pulau,provinsi,job_position,company_name,years_of_experience,years_of_experience_cat,average_processing_time,company_industry,company_size,work_environment_waktu_bekerja,work_environment_gaya_berpakaian,work_environment_gaya_berpakaian_contoh,work_environment_tunjangan,work_environment_bahasa,platform,pendidikan,jumlah_loker_per_provinsi
0,1,Jawa,Daerah Khusus Ibukota Jakarta,Spv Finance,PT Axindo Infotama,3,0 - 4 tahun,6,INDUSTRI PENGOLAHAN,51 - 200 pekerja,sabtu,formal,kemeja polos & dasi,"kesehatan, umum",Indonesia,jobstreet,D3,5985
1,2,Jawa,Daerah Khusus Ibukota Jakarta,Qualitative Research Executive,Perusahaan Dirahasiakan,3,0 - 4 tahun,tidak tersedia,TIDAK TERSEDIA,tidak spesifik,tidak tersedia,tidak spesifik,tidak spesifik,tidak tersedia,Tidak Dicantumkan,jobstreet,S1,5985
2,3,Jawa,Jawa Tengah,Field Collection Wonogiri,OTO Group,0,0 - 4 tahun,14,KEUANGAN DAN ASURANSI,lebih dari 5000 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, bonus",Indonesia,jobstreet,"D3, S1, SMA",548
3,4,Jawa,Daerah Khusus Ibukota Jakarta,Perawat Di Negara Jerman,PT Jerman Manajemen,1,0 - 4 tahun,11,PENDIDIKAN,51 - 200 pekerja,sabtu - kamis,formal,kemeja polos & dasi,kesehatan,Indonesia,jobstreet,D3,5985
4,5,Jawa,Jawa Tengah,Field Collection Pati,OTO Group,0,0 - 4 tahun,14,KEUANGAN DAN ASURANSI,lebih dari 5000 pekerja,senin - jumat,bisnis,kemeja,"kesehatan, bonus",Indonesia,jobstreet,"D3, S1, SMA",548


# Export csv

## Export data_loker

In [22]:
# export data master
# data.to_csv('data_master_16oct.csv', index=False)

In [23]:
# export data_loker
data.to_csv('data_loker_16oct_v2.csv', index=False)

In [30]:
data.shape

(12625, 18)

## Export data_pendidikan

In [24]:
# export pendidikan
# reformat data pendidikan menjadi csv baru
df_pendidikan = pd.DataFrame(columns=['id_loker', 'provinsi', 'pendidikan'])

for idx, row in data.iterrows():

    split_pendidikan = [data.strip().upper() for data in row['pendidikan'].split(',')]
    
    for pend in split_pendidikan:
        df_pendidikan = df_pendidikan.append({'id_loker': row['id_loker'],
                                              'provinsi': row['provinsi'],
                                              'pendidikan': pend}, ignore_index=True)

In [25]:
df_pendidikan.head()

,id_loker,provinsi,pendidikan
0,1,Daerah Khusus Ibukota Jakarta,D3
1,2,Daerah Khusus Ibukota Jakarta,S1
2,3,Jawa Tengah,D3
3,3,Jawa Tengah,S1
4,3,Jawa Tengah,SMA


In [26]:
# export data_pendidikan
df_pendidikan.to_csv('data_pendidikan_16oct_v2.csv', index=False)

## Export data_loker_prov

In [27]:
# export jumlah loker per provinsi
df_loker_prov = pd.DataFrame(columns=['id_loker', 'provinsi', 'jumlah_loker'])

for idx, row in data.iterrows():
    df_loker_prov = df_loker_prov.append({'id_loker': row['id_loker'],
                                          'provinsi': row['provinsi'],
                                          'jumlah_loker': row['jumlah_loker_per_provinsi']}, ignore_index=True)

In [28]:
df_loker_prov.head()

,id_loker,provinsi,jumlah_loker
0,1,Daerah Khusus Ibukota Jakarta,5985
1,2,Daerah Khusus Ibukota Jakarta,5985
2,3,Jawa Tengah,548
3,4,Daerah Khusus Ibukota Jakarta,5985
4,5,Jawa Tengah,548


In [29]:
# export data_loker_prov
df_loker_prov.to_csv('data_loker_prov_16oct_v2.csv', index=False)

## Export data_skill